In [1]:
import torch
import torchvision as tv
import numpy as np
import sys
import os
import random
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
class MovingFilterDataset(torch.utils.data.Dataset):
    def __init__(self, array, window, label, transform=None):
        self.array = array
        self.window = window
        self.label = label
        self.transform = transform
    
    def __getitem__(self, index):
        sample = np.moveaxis(self.array[index:index+self.window,...],0,-1)
        if self.transform:
            sample = self.transform(sample).float()
        return (sample, self.label)
    
    def __len__(self):
        return self.array.shape[0] - self.window

In [3]:
load_model = True

window_size = 20
num_epochs = 15
batch_size = 75
class_folders = [
            ['ball_data'],
            ['bott_data'],
            ['fist_data'],
            ['ind_data'],
            ['mid_data'],
            ['rin_data'],
            ['pin_data'],
            ['thr_data']
]

In [4]:
"""if load_data:
    print('loading training data as Dataset...')
    full_dataset = torch.load('rnn_dataset.pt')
else:
    full_dataset = []
    print('loading training data as Numpy and coverting...')
    for i,folder_list in enumerate(class_folders):
        class_files = [os.path.join(l,f) for l in folder_list for f in os.listdir(l) if f.endswith('-train-images.npy')]
        for im in class_files:
            print(im)
            im_data = np.load(im)
            print(im_data.shape)
            full_dataset.append(MovingFilterDataset(im_data,window_size,i,transform=tv.transforms.ToTensor()))
    print('concatenating datasets...')
    full_dataset = torch.utils.data.ConcatDataset(full_dataset)
    print('saving dataset for later...')
    #torch.save(full_dataset, 'rnn_dataset.pt')
#loader = torch.utils.data.DataLoader(full_dataset,batch_size=30,shuffle=True)"""

"if load_data:\n    print('loading training data as Dataset...')\n    full_dataset = torch.load('rnn_dataset.pt')\nelse:\n    full_dataset = []\n    print('loading training data as Numpy and coverting...')\n    for i,folder_list in enumerate(class_folders):\n        class_files = [os.path.join(l,f) for l in folder_list for f in os.listdir(l) if f.endswith('-train-images.npy')]\n        for im in class_files:\n            print(im)\n            im_data = np.load(im)\n            print(im_data.shape)\n            full_dataset.append(MovingFilterDataset(im_data,window_size,i,transform=tv.transforms.ToTensor()))\n    print('concatenating datasets...')\n    full_dataset = torch.utils.data.ConcatDataset(full_dataset)\n    print('saving dataset for later...')\n    #torch.save(full_dataset, 'rnn_dataset.pt')\n#loader = torch.utils.data.DataLoader(full_dataset,batch_size=30,shuffle=True)"

In [5]:
class MyModel1(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torch.nn.ModuleDict({
            'lstm': torch.nn.LSTM(636*256,32,batch_first=True),
            'linear': torch.nn.Linear(32,len(class_folders)),
            'sigmoid': torch.nn.Sigmoid()
        })
    def forward(self,x):
        out,_ = self.model['lstm'](x)
        out = self.model['linear'](out)
        out = self.model['sigmoid'](out)
        return out[:,-1,...]#.squeeze()
    def loss_function(self,x,y):
        output = self.forward(x)
        loss = torch.nn.CrossEntropyLoss()(output,y)
        return loss

In [6]:
class MyModel2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torch.nn.ModuleDict({
            'conv1': torch.nn.Conv2d(window_size,window_size,5,stride=2),
            'relu1': torch.nn.ReLU(),
            'conv2': torch.nn.Conv2d(window_size,window_size,5,stride=2),
            'relu2': torch.nn.ReLU(),
            'lstm': torch.nn.LSTM(9516,32,batch_first=True),
            'linear': torch.nn.Linear(32,len(class_folders)),
            'sigmoid': torch.nn.Sigmoid()
        })
    def forward(self,x):
        out = self.model['conv1'](x)
        out = self.model['relu1'](out)
        out = self.model['conv2'](out)
        out = self.model['relu2'](out)
        #out = torch.reshape(out,(batch_size,window_size,-1))
        out = torch.reshape(out,(*(out.size()[0:2]), -1))
        out,_ = self.model['lstm'](out)
        out = self.model['linear'](out)
        out = self.model['sigmoid'](out)
        return out[:,-1,...]
    def loss_function(self,x,y):
        output = self.forward(x)
        loss = torch.nn.CrossEntropyLoss()(output,y)
        return loss

In [99]:
network = MyModel2()
if load_model:
    network.load_state_dict(torch.load('rnn_network.pt'),strict=False)

optimizer = torch.optim.Adam(network.parameters(), lr=0.001)

In [101]:
print('loading training data as Numpy and coverting...')
files = [(i,os.path.join(d,f)) for i,l in enumerate(class_folders) for d in l for f in os.listdir(d) if f.endswith('-train-images.npy')]
random.shuffle(files)

losses = []
for i,im in files:
    print(im)
    im_data = np.load(im)
    print(im_data.shape)
    dataset = MovingFilterDataset(im_data,window_size,i,transform=tv.transforms.ToTensor())
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True,drop_last=True)
    file_loss = []
    for epoch in range(num_epochs):
        epoch_loss = []
        for batch,(seq,labels) in enumerate(dataloader):
            optimizer.zero_grad()
            loss = network.loss_function(seq,labels)
            loss.backward()
            optimizer.step()
            loss_val = loss.item()
            print('Epoch',epoch,'Batch',batch,'Loss:',loss_val,end='\r')
            epoch_loss.append(loss_val)
        file_loss.append(epoch_loss)
    losses.append(file_loss)
losses = np.array(losses)

loading training data as Numpy and coverting...
fist_data/FistRelax-200205-02-train-images.npy
(978, 636, 256)
ball_data/BallGrab-200205-01-train-images.npy
(978, 636, 256)
ind_data/IndFlex-200205-01-train-images.npy
(978, 636, 256)


KeyboardInterrupt: 

In [93]:
np.save('train_losses_new3.npy',losses)

In [102]:
print('loading testing data as Numpy and coverting...')
test_files = [(i,os.path.join(d,f)) for i,l in enumerate(class_folders) for d in l for f in os.listdir(d) if f.endswith('-test-images.npy')]
random.shuffle(test_files)

labels = []
preds = []
for i,im in test_files:
    print(im)
    im_data = np.load(im)
    print(im_data.shape)
    dataset = MovingFilterDataset(im_data,window_size,i,transform=tv.transforms.ToTensor())
    dataloader = torch.utils.data.DataLoader(dataset)
    for i,(seq,label) in enumerate(dataloader):
        pred = network(seq)
        #print(pred)
        pred = torch.argmax(pred)
        labels.append(label.item())
        preds.append(pred.item())

loading testing data as Numpy and coverting...
ball_data/BallGrab-200205-01-test-images.npy
(420, 636, 256)
fist_data/FistRelax-200205-01-test-images.npy
(420, 636, 256)
rin_data/RinFlex-200205-01-test-images.npy
(420, 636, 256)
ind_data/IndFlex-200205-01-test-images.npy
(420, 636, 256)
mid_data/MidFlex-200205-01-test-images.npy
(420, 636, 256)
thr_data/ThrExt-200205-01-test-images.npy
(420, 636, 256)
bott_data/BottGrab-200205-01-test-images.npy
(420, 636, 256)
pin_data/PinFlex-200205-01-test-images.npy
(420, 636, 256)
ball_data/BallGrab-200205-02-test-images.npy
(420, 636, 256)
ind_data/IndExt-200205-01-test-images.npy
(420, 636, 256)
fist_data/FistRelax-200205-02-test-images.npy
(420, 636, 256)


In [103]:
conf = confusion_matrix(labels,preds)
print(conf)

[[  0   0 800   0   0   0   0   0]
 [  0   0 400   0   0   0   0   0]
 [  0   0 800   0   0   0   0   0]
 [  0   0 800   0   0   0   0   0]
 [  0   0 400   0   0   0   0   0]
 [  0   0 400   0   0   0   0   0]
 [  0   0 400   0   0   0   0   0]
 [  0   0 400   0   0   0   0   0]]


In [97]:
plt.plot(losses.flatten())

In [52]:
torch.save(network.state_dict(),'rnn_network.pt')

In [ ]:
network = MyModel1()
network.load_state_dict(torch.load('rnn_network.pt',strict=False))

In [65]:
print('loading testing data as Numpy and coverting...')
test_files = [(i,os.path.join(d,f)) for i,l in enumerate(class_folders) for d in l for f in os.listdir(d) if f.endswith('-test-images.npy')]
random.shuffle(test_files)

loading testing data as Numpy and coverting...


In [66]:
test_files

[(5, 'rin_data/RinFlex-200205-01-test-images.npy'),
 (1, 'bott_data/BottGrab-200205-01-test-images.npy'),
 (2, 'fist_data/FistRelax-200205-02-test-images.npy'),
 (0, 'ball_data/BallGrab-200205-01-test-images.npy'),
 (3, 'ind_data/IndFlex-200205-01-test-images.npy'),
 (3, 'ind_data/IndExt-200205-01-test-images.npy'),
 (0, 'ball_data/BallGrab-200205-02-test-images.npy'),
 (2, 'fist_data/FistRelax-200205-01-test-images.npy'),
 (4, 'mid_data/MidFlex-200205-01-test-images.npy'),
 (7, 'thr_data/ThrExt-200205-01-test-images.npy'),
 (6, 'pin_data/PinFlex-200205-01-test-images.npy')]